In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json, os
from tqdm import tqdm
from pathlib import Path
from utils import *
import numpy as np
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Default paths
ROOT = Path("/content/drive/MyDrive/BDA Project/Amazon_products")
TRAIN_PATH = ROOT / "train"
TEST_PATH = ROOT / "test"

TRAIN_CORPUS_PATH = TRAIN_PATH / "train_corpus.txt"
TEST_CORPUS_PATH = TEST_PATH  / "test_corpus.txt"
CLASSES_PATH = ROOT / "classes.txt"
HIERARCHY_PATH = ROOT / "class_hierarchy.txt"
REL_KEYWORDS_PATH = ROOT / "class_related_keywords.txt"

In [ ]:
# ---------- Function for loads ----------

def load_lines(p: Path):
    with p.open("r", encoding="utf-8") as f:
        return [line.rstrip("\n") for line in f]

def load_pid2text(p: Path):
    """TSV: pid \\t text  -> dict[pid]=text"""
    pid2text = {}
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            parts = line.rstrip("\n").split("\t", 1)
            if len(parts) == 2:
                pid, text = parts
                pid2text[pid] = text
    return pid2text

def load_classes_int(p: Path):
    class_dict = {}
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            label_int, label_str = line.rstrip("\n").split("\t")
            class_dict[int(label_int)] = label_str
    return class_dict

def load_classes_str(p: Path):
    class_dict = {}
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            label_int, label_str = line.rstrip("\n").split("\t")
            class_dict[label_str] = int(label_int)
    return class_dict

def load_keywords(p: Path):
    keywords = {}
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            key, items = line.rstrip("\n").split(":")
            item_list = [item for item in items.split(",")]
            keywords[key] = item_list
    return keywords

def load_class_graph(p: Path):
    edges = []
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            p, c = map(int, line.rstrip("\n").split("\t"))
            edges.append((p, c))
    return edges

In [ ]:
# ---------- Read-only loads ----------

train_pid2text    = load_pid2text(TRAIN_CORPUS_PATH)
test_pid2text     = load_pid2text(TEST_CORPUS_PATH)
classes_int       = load_classes_int(CLASSES_PATH)
classes_str       = load_classes_str(CLASSES_PATH)
rel_keywords      = load_keywords(REL_KEYWORDS_PATH)
class_graph_edges = load_class_graph(HIERARCHY_PATH)

print(f"#train={len(train_pid2text):,}  #test={len(test_pid2text):,}")

#train=29,487  #test=19,658


## Using BERT to make embeddings

In [ ]:
# ===== BERT Tokenizer and Model =====
import torch
from transformers import AutoTokenizer, AutoModel

model_name = "Alibaba-NLP/gte-base-en-v1.5"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

configuration.py: 0.00B [00:00, ?B/s]

The repository Alibaba-NLP/gte-base-en-v1.5 references custom code contained in Alibaba-NLP/new-impl which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Alibaba-NLP/new-impl .
 You can inspect the repository content at https://hf.co/Alibaba-NLP/gte-base-en-v1.5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


modeling.py: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

NewModel(
  (embeddings): NewEmbeddings(
    (word_embeddings): Embedding(30528, 768, padding_idx=0)
    (rotary_emb): NTKScalingRotaryEmbedding()
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): NewEncoder(
    (layer): ModuleList(
      (0-11): 12 x NewLayer(
        (attention): NewSdpaAttention(
          (qkv_proj): Linear(in_features=768, out_features=2304, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (o_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): NewGatedMLP(
          (up_gate_proj): Linear(in_features=768, out_features=6144, bias=False)
          (down_proj): Linear(in_features=3072, out_features=768, bias=True)
          (act_fn): GELUActivation()
          (hidden_dropout): Dropout(p=0.1, inplace=False)
        )
        (attn_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (mlp_ln): LayerNorm((768,), eps=1

In [ ]:
def encode_texts(texts, batch_size=64, device=device):
    model.to(device)
    model.eval()
    all_embs = []

    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            enc = tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(device)
            out = model(**enc)
            # GTE 논문/리포 기준: CLS pooling 권장, 아니면 mean pooling도 가능
            token_emb = out.last_hidden_state        # (B, L, 768)
            attn = enc["attention_mask"].unsqueeze(-1)  # (B, L, 1)
            emb = (token_emb * attn).sum(dim=1) / attn.sum(dim=1)  # mean-pooling
            emb = F.normalize(emb, p=2, dim=1)
            all_embs.append(emb.cpu())
    return torch.cat(all_embs, dim=0)  # (N, 768)

In [ ]:
with torch.no_grad():
    train_embeddings = encode_texts(list(train_pid2text.values()))
    test_embeddings = encode_texts(list(test_pid2text.values()))

In [ ]:
# 새 셀: 생성된 BERT 임베딩을 JSON 파일로 저장

EMBEDDNIGS_SAVE_DIR = Path("/content/drive/MyDrive/BDA Project/Embeddings3")

os.makedirs(EMBEDDINGS_SAVE_DIR, exist_ok=True)

# train_embeddings, test_embeddings 변수와 pid 순서가 존재해야 합니다.
# train_pid2text, test_pid2text의 keys() 순서가 encode_texts 호출 순서와 일치하는지 확인하세요.
train_pids = list(train_pid2text.keys())
test_pids = list(test_pid2text.keys())

# numpy list 변환 (메모리 주의: 대규모일 경우 파일 크기 큼)
train_arr = train_embeddings.detach().numpy() if isinstance(train_embeddings, torch.Tensor) else np.array(train_embeddings)
test_arr  = test_embeddings.detach().numpy()  if isinstance(test_embeddings, torch.Tensor)  else np.array(test_embeddings)

# pid -> embedding(리스트) 매핑 생성 및 저장
train_map = {pid: emb.tolist() for pid, emb in zip(train_pids, train_arr)}
test_map  = {pid: emb.tolist() for pid, emb in zip(test_pids,  test_arr)}

with open(EMBEDDINGS_SAVE_DIR / "train_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(train_map, f, ensure_ascii=False)

with open(EMBEDDINGS_SAVE_DIR / "test_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(test_map, f, ensure_ascii=False)

print(f"Saved train_embeddings -> {EMBEDDINGS_SAVE_DIR}/train_embeddings.json (items={len(train_map)})")
print(f"Saved test_embeddings  -> {EMBEDDINGS_SAVE_DIR}/test_embeddings.json  (items={len(test_map)})")

Saved train_embeddings -> /content/drive/MyDrive/BDA Project/Embeddings3/train_embeddings.json (items=29487)
Saved test_embeddings  -> /content/drive/MyDrive/BDA Project/Embeddings3/test_embeddings.json  (items=19658)
Saved class_embeddings  -> /content/drive/MyDrive/BDA Project/Embeddings3/class_embeddings.json  (items=531)


In [ ]:
class_texts = []
for i in range(len(list(classes_int.values()))):
  class_i = classes_int[i]
  text = class_i + " : " + ", ".join(rel_keywords[class_i])
  class_texts.append(text)

In [ ]:
with torch.no_grad():
    class_embeddings = encode_texts(class_texts)

In [ ]:
EMBEDDINGS_SAVE_DIR = Path("/content/drive/MyDrive/BDA Project/Embeddings3")

class_pids = list(classes_int.keys())

# numpy list 변환 (메모리 주의: 대규모일 경우 파일 크기 큼)
class_arr = class_embeddings.detach().numpy() if isinstance(class_embeddings, torch.Tensor) else np.array(class_embeddings)

# pid -> embedding(리스트) 매핑 생성 및 저장
class_map = {pid: emb.tolist() for pid, emb in zip(class_pids, class_arr)}

with open(EMBEDDINGS_SAVE_DIR / "class_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(class_map, f, ensure_ascii=False)

print(f"Saved class_embeddings  -> {EMBEDDINGS_SAVE_DIR}/class_embeddings.json  (items={len(class_map)})")

Saved class_embeddings  -> /content/drive/MyDrive/BDA Project/Embeddings3/class_embeddings.json  (items=531)
